# Cloud Data Transfer Speeds Benchmarking Workflow

Add overview of workflow

## Step 1: Define Workflow Inputs

Add more detailed explanation of workflow inputs. More inputs will be added as workflow development progresses.

### Computing Resource Information

Input the names of as many resources as you wish to run the benchmarks on. The name(s) of your resource(s) can be found on the **`RESOURCES`** tab in the Parallel Works platform.

Sample Input:

`resources = ["<resource-1-name>", "<resource-2-name>", ..., "<resource-n-name>"]`

In [27]:
resources = ["gcptest"]

### Randomly Generated File Options

First element determines if the file will be generated, second element determines the file size.


**NOTE: The second element does not matter if the first is set to "False", but it still needs to be included**

Sample Input:

`filetype = [<True/False>, <Desired size of generated file (in GB)>]`

In [ ]:
csv = [True, 1]
netcdf = [False, 0]
binary = [False, 0]

## Step 2: Notebook Setup

In [3]:
import os
import sys
import json

os.environ["resources"] = json.dumps(resources)  

filetype = list(map(str, [csv[0], netcdf[0], binary[0]]))
filesize = list(map(str, [csv[1], netcdf[1], binary[1]]))
os.environ["randgen_files"] = json.dumps(filetype)
os.environ["randgen_sizes"] = json.dumps(filesize)
#os.environ["FILESIZE"] = "1" # The desired file sizes (IN GB) for randomly generated files

gcptest
